In [118]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyproj import Proj, transform

plt.rcParams['font.family'] = 'Malgun Gothic'

In [119]:
#위도, 경도
jeonnam_info=pd.read_excel('./기본정보_위경도.xlsx')
jeonnam_info.drop('읍면동/구',axis=1)

#숙박 유형
pension=pd.read_excel("./숙박유형_관광펜션업.xlsx")
camping=pd.read_excel("./숙박유형_일반야영장업.xlsx")
hanok=pd.read_excel("./숙박유형_한옥체험업.xlsx")
resort=pd.read_excel("./숙박유형_호텔리조트.xlsx") 
rural=pd.read_excel('./숙박유형_농어촌숙박.xlsx')

#비대면
alien1=pd.read_csv('./비대면_외지인방문자수_1.csv',encoding='cp949')
alien2=pd.read_csv('./비대면_외지인방문자수_2.csv',encoding='cp949')
resider1=pd.read_csv('./비대면_현지인방문자수_1.csv',encoding='cp949')
resider2=pd.read_csv('./비대면_현지인방문자수_2.csv',encoding='cp949')

#식음및부대시설
meal=pd.read_csv('./부대시설_음식점.csv',encoding='cp949')
cafe=pd.read_csv('./부대시설_휴게음식점.csv',encoding='utf-8')

#콘텐츠
contents1=pd.read_csv('./콘텐츠_관광지_검색순위_상.csv',encoding='cp949')
contents2=pd.read_csv('./콘텐츠_관광지_검색순위_하.csv',encoding='cp949')

숙박유형

In [120]:
pension.drop(pension[(pension['영업상태구분코드']!=1)].index,inplace=True)
camping.drop(camping[(camping['영업상태구분코드']!=1)].index,inplace=True)
hanok.drop(hanok[(hanok['영업상태구분코드']!=1)].index,inplace=True)
resort.drop(resort[(resort['영업상태구분코드']!=1)].index,inplace=True)

In [121]:
pension=pension[pension['개방자치단체코드'].between(4800000,5100000)]
camping=camping[camping['개방자치단체코드'].between(4800000,5100000)]
hanok=hanok[hanok['개방자치단체코드'].between(4800000,5100000)]
resort=resort[resort['개방자치단체코드'].between(4800000,5100000)]

In [122]:
ps=pension.loc[:,['좌표정보(X)','좌표정보(Y)']]
cp=camping.loc[:,['좌표정보(X)','좌표정보(Y)']]
hk=hanok.loc[:,['좌표정보(X)','좌표정보(Y)']]
rs=resort.loc[:,['좌표정보(X)','좌표정보(Y)']]
rr=rural.loc[:,['좌표정보(X)','좌표정보(Y)']]

In [123]:
proj_1 = Proj(init='epsg:2097')
proj_2 = Proj(init='epsg:4326')

accomm=[ps,cp,hk,rs,rr]

for acco in accomm:
    convert = transform(proj_1, proj_2, acco['좌표정보(X)'].values, acco['좌표정보(Y)'].values)
    acco['경도'] = convert[0]
    acco['위도'] = convert[1]
    acco=acco.loc[:,['위도','경도']]

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\user\AppData\Local\Temp\ipykernel_176288\331578052.py:7: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas

비대면 지수

In [125]:
alien=pd.concat([alien1,alien2],axis=1,ignore_index=True)
alien.drop([2],axis=1,inplace=True)
alien['평균외지인방문자수']=(alien[1]+alien[3])/2
regions=alien[0]

resider=pd.concat([resider1,resider2],axis=1,ignore_index=True)
resider.drop([2],axis=1,inplace=True)
resider['평균현지인방문자수']=(resider[1]+resider[3])/2

calm_score=alien['평균외지인방문자수']/resider['평균현지인방문자수']*100 #선행연구 참고
induce_score=alien['평균외지인방문자수']-resider['평균현지인방문자수']

attract_df=pd.concat([regions,calm_score,induce_score],axis=1)
attract_col=['지역','비대면지수','유입지수']
attract_df.columns=attract_col

print(attract_df)

     지역       비대면지수        유입지수
0   목포시   41.232053 -36120715.5
1   여수시   49.405687 -27693993.5
2   순천시   43.061186 -34299431.0
3   나주시   62.008706 -10008175.0
4   광양시   49.087961 -14126210.0
5   담양군  164.141395   6075746.5
6   곡성군  111.218072    574927.5
7   구례군  117.905852    904697.0
8   고흥군   69.595750  -2972475.0
9   보성군   98.632132    -98513.0
10  화순군   84.238047  -2341198.5
11  장흥군   58.054895  -3816795.5
12  강진군   70.229029  -2399610.5
13  해남군   73.605863  -3163934.0
14  영암군   80.345210  -2547058.5
15  무안군   79.961388  -3741836.0
16  함평군  136.376471   1713952.0
17  영광군   62.530661  -4386753.0
18  장성군  124.785514   2058376.5
19  완도군   52.339439  -5384338.0
20  진도군   90.278926   -538942.0
21  신안군  122.562591   1275741.0


콘텐츠

In [126]:
contents1.info()

contents2.drop('광역시/도',axis=1,inplace=True)
contents2

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   순위        500 non-null    int64 
 1   시/군/구     500 non-null    object
 2   관광지명      500 non-null    object
 3   도로명주소     500 non-null    object
 4   중분류 카테고리  500 non-null    object
 5   소분류 카테고리  500 non-null    object
 6   검색건수      500 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 27.5+ KB


,순위,시/군/구,관광지명,도로명주소,중분류 카테고리,소분류 카테고리,검색건수
0,1,순천시,순천만국가정원,전남 순천시 국가정원1호길 47-0,기타관광,기타관광,245254
1,2,목포시,목포역,전남 목포시 영산로 98-0,기타관광,교통시설,159899
2,3,순천시,순천역,전남 순천시 팔마로 135-0,기타관광,교통시설,141882
3,4,담양군,죽녹원,전남 담양군 죽녹원로 119-0,자연관광,자연공원,132973
4,5,여수시,오동도,전남 여수시 오동도로 222-0,자연관광,자연경관(하천/해양),113284
...,...,...,...,...,...,...,...
495,496,화순군,백아산눈썰매장,전남 화순군 백아로 0-0,레저스포츠,육상레저스포츠,2556
496,497,여수시,대경도선착장,전남 여수시 대경도길 6-0,기타관광,교통시설,2548
497,498,화순군,화순고인돌공원,전남 화순군,문화관광,도시공원,2547
498,499,나주시,태평사,전남 나주시 금성산길 387-83,역사관광,종교성지,2538


In [150]:
count_num=0
sites=[]

for i in range(len(contents2)): #22'09~23'08
   for j in range(len(contents1)): #21'09~22'08
        if contents2['관광지명'][i]==contents1['관광지명'][j]:
            count_num+=1
            sites.append(contents2['관광지명'][i])

print(count_num) #451: 인기 관광지 500곳 중 451곳은 반복해서 수요가 많다.
print(sites)

451
['순천만국가정원', '목포역', '순천역', '죽녹원', '오동도', '여수해상케이블카놀아정류장', 'LF스퀘어광양점', '완도항여객선터미널', '향일암', '목포해상케이블카북항승강장', '아쿠아플라넷여수', '여수엑스포역', '목포종합버스터미널', '화엄사', '나주역', '여수예술랜드', '목포연안여객선터미널', '녹동항', '롯데아울렛남악점', '백양사', '여수공항', '메타프로방스', '순천종합버스터미널', '순천만습지', '낙안읍성민속마을', '섬진강기차마을', '유월드루지테마파크', '율포해수욕장', '영광종합버스터미널', '선암사', '아르떼뮤지엄여수', '불갑사', '송광사', '진도항', '녹동신항연안여객선터미널', 'NC백화점순천점', '여수수산시장', '돌산공원', '무안CC', 'CGV순천신대', '장성역', '푸른솔GC', '무안국제공항', '호남해안로드4코스영광백수해안도로', '여수수산물특화시장', '대흥사', '벌교역', '목포항국제여객터미널', '골드레이크CC', '해남종합버스터미널', '담양관방제림', '사성암', '여수종합버스터미널', '여천역', '암태남강선착장', '중마버스터미널', '신지명사십리해수욕장', '무등산CC', '만성리검은모래해변', '메타세쿼이아가로수길', '구례구역', '순천아랫장', '운림산방', '순천오픈세트장', '아크로CC', '여수연안여객선터미널', '정남진장흥토요시장', '강진버스여객터미널', '신기항여객터미널', 'JNJ골프리조트', '소쇄원', '갓바위', '함평엘리체CC', '나로우주센터우주과학관', '천은사', '진남관공사중(2023년12월완공예정)', '땅끝전망대', '골프존카운티순천', '돌머리해변', '무안시외버스터미널', '목포북항', '땅끝선착장', '여수구항방파제등대', '목포근대역사관1관', '청호시장', '다산베아채CC', '파인비치골프링크스', '목포종합수산시장', '화순CC', '웅천친수공원', '광주CC', '메타세콰이어길', '사우스링스영암CC짐앵', '파인힐

      순위 시/군/구   관광지명           도로명주소 중분류 카테고리 소분류 카테고리  검색건수
499  499   무안군  도원선착장  전남 무안군 운해로 0-0     기타관광     교통시설  2538


시각화

In [ ]:
xmin=min(jeonnam_info['경도'])
xmax=max(jeonnam_info['경도'])
ymin=min(jeonnam_info['위도'])
ymax=max(jeonnam_info['위도'])

In [ ]:
#숙소유형
plt.scatter(x=ps['경도'],y=ps['위도'],alpha=0.2,s=100,c='green')
plt.scatter(x=cp['경도'],y=cp['위도'],alpha=0.2,s=100,c='green')
plt.scatter(x=hk['경도'],y=hk['위도'],alpha=0.2,s=100,c='green')
plt.scatter(x=rs['경도'],y=rs['위도'],alpha=0.2,s=100,c='green')
plt.scatter(x=rr['경도'],y=rr['위도'],alpha=0.2,s=100,c='green')

plt.scatter(x=mea['경도'],y=restuarant['위도'],alpha=0.5,s=100,c='m',label='식당')

plt.title('숙소유형 분포도')

plt.xlim(xmin,xmax)
plt.ylim(ymin,ymax)

plt.legend()

plt.show()

print(xmin,xmax,ymin,ymax)